In [ ]:
import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd
from matplotlib.pyplot import subplots
import matplotlib.dates as mdates

load_dotenv()

database_password = os.environ.get("DATABASE_PASSWORD")
database_username = os.environ.get("DATABASE_USERNAME")
database_host = os.environ.get("DATABASE_HOST")
database_port = os.environ.get("DATABASE_PORT")
database_name = os.environ.get("DATABASE_NAME")

connection = psycopg2.connect(database=database_name,
                        host=database_host,
                        user=database_username,
                        password=database_password,
                        port=database_port)

In [ ]:
def sql_to_dataframe(conn, query) -> pd.DataFrame:
    """
    Import data from a PostgreSQL database using a SELECT query.
    Based on https://medium.com/@alestamm/importing-data-from-a-postgresql-database-to-a-pandas-dataframe-5f4bffcd8bb2
    """
    cursor = conn.cursor()
    try:
        cursor.execute(query)
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}”")
        cursor.close()
        return 1
    # The execute returns a list of tuples:
    tuples_list = cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]
    cursor.close()
    # Now we need to transform the list into a pandas DataFrame:
    df = pd.DataFrame(tuples_list, columns=colnames)
    return df

# Jobs live, including historic

In [ ]:
jobs_live_df = sql_to_dataframe(conn=connection, query="""SELECT DATE(checked_time), COUNT(DISTINCT scrape_id)
	FROM (SELECT * FROM sitemap_entries
		WHERE checked_time > '2024-01-10' AND NOT checked_time = '2024-02-10 01:36:09.964627+00' AND NOT url = 'https://www.civilservicejobs.service.gov.uk/csr/index.cgi'
		ORDER BY checked_time ASC) m
GROUP BY DATE(checked_time)""")

In [ ]:
jobs_live_df['date'] = pd.to_datetime(jobs_live_df['date'])
jobs_live_df.set_index("date", inplace=True)

In [ ]:
historic_scrapes = pd.read_csv("../src/data/historic_scrapes.csv")

In [ ]:
historic_scrapes["date"] = pd.to_datetime(historic_scrapes["date"], dayfirst=True)

In [ ]:
historic_scrapes.set_index("date", inplace=True)

In [ ]:
jobs_live_df

In [ ]:
historic_scrapes

In [ ]:
combined_df = pd.concat([jobs_live_df, historic_scrapes])

In [ ]:
combined_df = combined_df.sort_index()

In [ ]:
combined_df.head(20)

In [ ]:
fig, ax = subplots(figsize=(12,6))

ax.plot(combined_df.index, combined_df["count"])

ax.set_ylim(bottom=0)

ax.set_title("Number of jobs live on Civil Service Jobs")

ax.set_ylabel("Number of jobs live")

#ax.set_xticks(combined_df["date"])
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.tick_params(axis='x', rotation=70)

fig.subplots_adjust(bottom=0.27)

fig.text(0.5, 0.01, "[Quick, non peer-reviewed visualisation based on unofficial once-daily web scraping since 2024-01-11. It may be incorrect. \n Measurements before then obtained using Archive.org. \n Only 'external' jobs (available to all, not only existing civil servants) are included. \n Civil Service Jobs data is available under the Open Government Licence v3.0]", ha="center")

# Jobs live on Wednesdays

In [ ]:
jobs_live_df = sql_to_dataframe(conn=connection, query="""SELECT DATE(checked_time), COUNT(DISTINCT scrape_id)
	FROM (SELECT * FROM sitemap_entries
		WHERE checked_time > '2024-01-10' AND NOT checked_time = '2024-02-10 01:36:09.964627+00' AND NOT url = 'https://www.civilservicejobs.service.gov.uk/csr/index.cgi'
		ORDER BY checked_time ASC) m
GROUP BY DATE(checked_time)""")

In [ ]:
jobs_live_df['date'] = pd.to_datetime(jobs_live_df['date'])

# Filter the DataFrame to only include Wednesdays (where dayofweek == 2)
wednesdays_df = jobs_live_df[jobs_live_df['date'].dt.dayofweek == 2]

In [ ]:
fig, ax = subplots(figsize=(10,6))

ax.plot(wednesdays_df["date"], wednesdays_df["count"], marker="+", markeredgecolor="red")

ax.set_ylim(bottom=0)

ax.set_title("Number of jobs live on Civil Service Jobs")

ax.set_ylabel("Number of jobs live")

ax.set_xticks(wednesdays_df["date"])
ax.tick_params(axis='x', rotation=70)

fig.subplots_adjust(bottom=0.28)

fig.text(0.5, 0.01, "[Quick, non peer-reviewed visualisation based on unofficial once-daily web scraping. \n Only 'external' jobs (available to all, not only existing civil servants) are included. \n Civil Service Jobs data is available under the Open Government Licence v3.0]", ha="center")

# Total new jobs added per day

In [ ]:
earliest_times_per_jcode_df = sql_to_dataframe(conn=connection, query="""SELECT url, COUNT(DISTINCT scrape_id), MIN(checked_time) AS earliest_checked_time, MIN(updated_time) AS earliest_updated_time 
                                               FROM sitemap_entries 
                                               WHERE checked_time > '2024-01-10' AND NOT checked_time = '2024-02-10 01:36:09.964627+00' AND NOT url = 'https://www.civilservicejobs.service.gov.uk/csr/index.cgi'
                                               GROUP BY url""")

In [ ]:
earliest_times_per_jcode_df["earliest_updated_time"] = pd.to_datetime(earliest_times_per_jcode_df["earliest_updated_time"], utc=True)

df_counts_by_date = earliest_times_per_jcode_df.groupby(earliest_times_per_jcode_df["earliest_updated_time"].dt.date).size().reset_index(name="count")

In [ ]:
df_counts_by_date["earliest_updated_time"] = pd.to_datetime(df_counts_by_date["earliest_updated_time"])
df_counts_by_date = df_counts_by_date[df_counts_by_date["earliest_updated_time"] >= pd.to_datetime("2024-01-11")]
df_counts_by_date.set_index("earliest_updated_time", inplace=True)

In [ ]:
df_counts_by_date = df_counts_by_date.asfreq('D', fill_value=0)

In [ ]:
fig, ax = subplots(figsize=(10,6))

ax.plot(df_counts_by_date.index, df_counts_by_date["count"])

ax.set_ylim(bottom=0)

ax.set_title("Number of new jobs posted on Civil Service Jobs per day")

ax.set_ylabel("Number of new jobs posted")

fig.subplots_adjust(bottom=0.18)

fig.text(0.5, 0.01, "[Quick, non peer-reviewed visualisation based on unofficial once-daily web scraping. \n Only 'external' jobs (available to all, not only existing civil servants) are included. \n Civil Service Jobs data is available under the Open Government Licence v3.0]", ha="center")

# Total new jobs added per week

In [ ]:
earliest_times_per_jcode_df_for_week = sql_to_dataframe(conn=connection, query="""SELECT url, COUNT(DISTINCT scrape_id), MIN(checked_time) AS earliest_checked_time, MIN(updated_time) AS earliest_updated_time 
                                               FROM sitemap_entries 
                                               WHERE checked_time > '2024-01-10' AND NOT checked_time = '2024-02-10 01:36:09.964627+00' AND NOT url = 'https://www.civilservicejobs.service.gov.uk/csr/index.cgi'
                                               GROUP BY url""")

In [ ]:
earliest_times_per_jcode_df_for_week["earliest_updated_time"] = pd.to_datetime(earliest_times_per_jcode_df_for_week["earliest_updated_time"], utc=True)
df_counts_by_week = earliest_times_per_jcode_df_for_week.groupby(earliest_times_per_jcode_df_for_week["earliest_updated_time"].dt.to_period('W')).size().to_timestamp().reset_index(name="count")

In [ ]:
df_counts_by_week["earliest_updated_time"] = pd.to_datetime(df_counts_by_week["earliest_updated_time"])
df_counts_by_week = df_counts_by_week[df_counts_by_week["earliest_updated_time"] >= pd.to_datetime("2024-01-15")]
df_counts_by_week = df_counts_by_week[df_counts_by_week["earliest_updated_time"] <= pd.to_datetime("2024-05-19")] # Set to the Sunday at the end of last week want to include
df_counts_by_week.set_index("earliest_updated_time", inplace=True)

In [ ]:
df_counts_by_week = df_counts_by_week.asfreq('W-MON', fill_value=0)

In [ ]:
fig, ax = subplots(figsize=(16,6))

ax.plot(df_counts_by_week.index, df_counts_by_week["count"], marker="+", markeredgecolor="red")

ax.set_ylim(bottom=0)

ax.set_title("Number of new jobs posted on Civil Service Jobs per week")

ax.set_ylabel("Number of new jobs posted")

ax.set_xticks(df_counts_by_week.index)

n = 2  # Keeps every nth label. From https://stackoverflow.com/questions/20337664/cleanest-way-to-hide-every-nth-tick-label-in-matplotlib-colorbar
[l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % n != 0]

fig.subplots_adjust(bottom=0.18)

fig.text(0.5, 0.01, "(Quick, non peer-reviewed visualisation based on unofficial once-daily web scraping. \n Only 'external' jobs (available to all, not only existing civil servants) are included. \n Civil Service Jobs data is available under the Open Government Licence v3.0)", ha="center")

# Jobs live on Wednesdays, for a specific department

In [ ]:
department_name = "Department for Environment, Food and Rural Affairs"

jobs_live_for_dept_df = sql_to_dataframe(conn=connection, query=f"""SELECT
	DATE (CHECKED_TIME),
	COUNT(DISTINCT URL)
FROM
	(
		SELECT
			MAIN.URL,
			MAIN.CHECKED_TIME,
			DEPARTMENTS.DEPARTMENT_NAME
		FROM
			(
				SELECT
					FILTERED_SITEMAP_ENTRIES.*,
					DEPARTMENT_ID_FOR_EACH_SCRAPE_ID.DEPARTMENT_ID
				FROM
					(
						SELECT
							*
						FROM
							SITEMAP_ENTRIES
						WHERE
							CHECKED_TIME > '2024-01-10'
							AND NOT CHECKED_TIME = '2024-02-10 01:36:09.964627+00'
							AND NOT URL = 'https://www.civilservicejobs.service.gov.uk/csr/index.cgi'
					) FILTERED_SITEMAP_ENTRIES
					LEFT JOIN (
						SELECT
							SCRAPE_ID,
							DEPARTMENT_ID
						FROM
							SCRAPES_EXTRACTED
					) DEPARTMENT_ID_FOR_EACH_SCRAPE_ID ON FILTERED_SITEMAP_ENTRIES.SCRAPE_ID = DEPARTMENT_ID_FOR_EACH_SCRAPE_ID.SCRAPE_ID
			) MAIN
			LEFT JOIN DEPARTMENTS ON MAIN.DEPARTMENT_ID = DEPARTMENTS.DEPARTMENT_ID
		WHERE
			DEPARTMENT_NAME = '{department_name}'
	) SITEMAP_ENTRIES_WITH_DEPARTMENT_NAME
GROUP BY
	DATE (CHECKED_TIME)""")

jobs_live_for_dept_df['date'] = pd.to_datetime(jobs_live_for_dept_df['date'])

jobs_live_for_dept_df.set_index("date", inplace=True)
jobs_live_for_dept_df = jobs_live_for_dept_df.asfreq("D", fill_value=0)

# Filter the DataFrame to only include Wednesdays (where dayofweek == 2)
jobs_live_for_dept_wednesdays_df = jobs_live_for_dept_df[jobs_live_for_dept_df.index.dayofweek == 3]

fig, ax = subplots(figsize=(10,6))

ax.plot(jobs_live_for_dept_wednesdays_df.index, jobs_live_for_dept_wednesdays_df["count"], marker="+", markeredgecolor="red")

ax.set_ylim(bottom=0)

ax.set_title(f"Number of jobs live on Civil Service Jobs from '{department_name}'")

ax.set_ylabel("Number of jobs live")

ax.set_xticks(jobs_live_for_dept_wednesdays_df.index)
ax.tick_params(axis='x', rotation=70)

fig.subplots_adjust(bottom=0.35)

fig.text(0.5, 0.01, "As recorded at 1AM. Dates plotted are Thursdays. \n\n[This is a quick, non peer-reviewed visualisation based on unofficial once-daily web scraping. It may be incorrect. \n Only 'external' jobs (available to all, not only existing civil servants) are included. \n Civil Service Jobs data is available under the Open Government Licence v3.0]", ha="center")